In [2]:
from bs4 import BeautifulSoup
import requests
import json


In [3]:
with open('../../resource/USUK_artist.json') as user_file:
    file_contents = user_file.read()
    parsed_json = set(json.loads(file_contents))
    newList = set()

In [4]:
def getData(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        page_content = response.text
        soup = BeautifulSoup(page_content, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

page_content = ""

def get_all_similar_artist(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        page_content = response.text
        soup = BeautifulSoup(page_content, 'html.parser')
        similar_artists = soup.select(".similar-artists-item-name > a")
        for artist in similar_artists:
            if "," in artist or "&" in artist:
                continue
            newList.add(artist.get_text().strip())
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

def findSimilarArtist(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        global page_content 
        page_content = response.text
        soup = BeautifulSoup(page_content, 'html.parser')
        last_page = int(soup.find_all(attrs={"class": "pagination-page"})[-1].find('a').get_text().strip())
        for i in range(1, last_page+1):
            get_all_similar_artist(f"{url}?page={i}")
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
for artist in parsed_json.copy():
    print(artist)
    newList.add(artist)
    artist_queue = artist.replace(" ", "+")
    findSimilarArtist(f"https://www.last.fm/music/{artist_queue}/+similar")
    parsed_json.remove(artist)

In [6]:
len(newList)

241

In [7]:
newList.update(parsed_json)
newList

{'.38 Special',
 'Aerosmith',
 'Aimee Mann',
 'Al Stewart',
 'America',
 'Asia',
 'Bachman-Turner Overdrive',
 'Bad Company',
 'Badfinger',
 'Beck',
 'Ben Harper',
 'Big Star',
 'Billy Joel',
 'Billy Squier',
 'Blind Faith',
 'Blue Öyster Cult',
 'Bob Dylan',
 'Bob Dylan and The Band',
 'Bob Seger',
 'Bob Seger & The Silver Bullet Band',
 'Boston',
 'Boz Scaggs',
 'Brandi Carlile',
 'Bruce Hornsby',
 'Bruce Hornsby & the Range',
 'Bruce Springsteen',
 'Bruce Springsteen & The E Street Band',
 'Bryan Adams',
 'Buffalo Springfield',
 'Carole King',
 'Cat Stevens',
 'Cheap Trick',
 'Chicago',
 'Chris Isaak',
 'Counting Crows',
 'Creedence Clearwater Revival',
 'Crosby & Nash',
 'Crosby, Stills & Nash',
 'Crosby, Stills, Nash & Young',
 'Crowded House',
 'Dan Fogelberg',
 'Dave Edmunds',
 'Dave Mason',
 'Dave Matthews Band',
 'David Crosby',
 'David Gilmour',
 'David Lee Roth',
 'Def Leppard',
 'Derek and the Dominos',
 'Dire Straits',
 'Don Henley',
 'Don McLean',
 'Donald Fagen',
 'Eagle

In [ ]:
result = filter(lambda artist: ',' not in artist, newList)
result = filter(lambda artist: '&' not in artist, result)
result = list(result)

In [ ]:
len(result)

In [ ]:
result.sort()
with open("artistList.json", "w") as outfile:
    json.dump(result, outfile)